In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torchsummary import summary

print(torch.__version__)

In [ ]:
# Hyper-parameters
learning_rate = 0.001
n_epochs = 20
batch_size = 100
n_classes = 10
random_seed = 777

In [ ]:
# For Reproducibilty
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
# Data 불러오기
data_dir = './data'

train_data = datasets.MNIST(
    root=data_dir,
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

val_data = datasets.MNIST(
    root=data_dir,
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [ ]:
train_data

In [ ]:
val_data

In [ ]:
label_list = train_data.classes
label_list

In [ ]:
# Data Visualization
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label_list[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
# DataLoader 만들기
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [ ]:
# DataLoader를 통해 반복하기(iterate)
# 이미지와 정답(label)을 표시합니다.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
# CNN Model 만들기
class VanillaCNN(nn.Module):
  def __init__(self):
    super(VanillaCNN, self).__init__()
    self.conv1 = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1),
                               nn.ReLU(),
                               nn.MaxPool2d(2,2))
    self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),
                               nn.ReLU(),
                               nn.MaxPool2d(2,2))
    self.conv3 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
                               nn.ReLU(),
                               nn.MaxPool2d(2,2))
    self.fc1 = nn.Sequential(nn.Linear(3*3*128, 256),
                             nn.ReLU(),
                             nn.Dropout(0.4))
    self.fc2 = nn.Sequential(nn.Linear(256, 10))

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.view(-1, 3*3*128)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [ ]:
# Model instance 생성, device 설정
model = VanillaCNN().to(device)

In [ ]:
print(model)

In [ ]:
summary(model, input_size=(1,28,28))

In [ ]:
# Loss Function
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Optimizer
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [ ]:
# Train Function
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  n_batches = len(dataloader)
  model.train()
  train_loss = 0.
  for batch, (data, label) in enumerate(dataloader):
    data, label = data.to(device), label.to(device)
    pred = model(data)
    loss = loss_fn(pred, label)    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
  train_loss /= n_batches

  print(f"Training Loss: {train_loss:>8f}")

In [ ]:
# Validation Function
def val_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  n_batches = len(dataloader)
  model.eval()
  val_loss, correct = 0., 0
  with torch.no_grad():
    for data, label in dataloader:
      data, label = data.to(device), label.to(device)
      pred = model(data)
      val_loss += loss_fn(pred, label).item()
      correct += (pred.argmax(1) == label).type(torch.float).sum().item()
  val_loss /= n_batches
  correct /= size
  print(f"Validation Loss: {val_loss:>8f} Validation Accuracy: {(100*correct):>0.4f}%, \n")

In [ ]:
# 학습 진행하기
for epoch in range(n_epochs):
  print(f"<<Epoch {epoch+1}>>\n------------------------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  val_loop(val_dataloader, model, loss_fn)
print("Training Done!")

In [ ]:
# 학습된 Model 저장하기 - weight only
save_path = './model_weights.pth'
torch.save(model.state_dict(), save_path)

In [ ]:
# 새로운 Model에 저장된 weight load하기
model2 = VanillaCNN().to(device)

In [ ]:
model2.eval()
val_loop(val_dataloader, model2, loss_fn)

In [ ]:
model2.load_state_dict(torch.load(save_path))
val_loop(val_dataloader, model2, loss_fn)

In [ ]:
# 학습된 Model 저장하기 - model 전체 저장
model_save_path = './model.pth'
torch.save(model, model_save_path)

In [ ]:
# 불러오기
model3 = torch.load(model_save_path)

In [ ]:
model3.eval()
val_loop(val_dataloader, model3, loss_fn)